# 라이브러리 호출

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.preprocessing import StandardScaler, MinMaxScaler # 스케일링
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT
from nltk.corpus import names # nltk
import nltk
nltk.download("names")
from nltk import NaiveBayesClassifier
from scipy import stats
from collections import Counter
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# cv스태밍
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

[nltk_data] Downloading package names to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!


# 데이터로딩

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

In [3]:
test = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Test')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0013_01,Earth,True,G,3.0,G3,S,G/3/S,TRAPPIST-1e,27,False,0,0,0,0,0,Nelly Carsoning,NaN
1,0018_01,Earth,False,F,4.0,F4,S,F/4/S,TRAPPIST-1e,19,False,0,9,0,2823,0,Lerome Peckers,NaN
2,0019_01,Europa,True,C,0.0,C0,S,C/0/S,55 Cancri e,31,False,0,0,0,0,0,Sabih Unhearfus,NaN
3,0021_01,Europa,False,C,1.0,C1,S,C/1/S,TRAPPIST-1e,38,False,0,6652,0,181,585,Meratz Caltilter,NaN
4,0023_01,Earth,False,F,5.0,F5,S,F/5/S,TRAPPIST-1e,20,False,10,0,635,0,0,Brence Harperez,NaN


# 전처리

## 이상치 확인 및 제거

In [4]:
def outlier_detection_train(df, n, columns):
    rows = []
    will_drop_train = []
    for col in columns:
        Q1 = np.nanpercentile(data[col], 25)
        Q3 = np.nanpercentile(data[col], 75)
        IQR = Q3 - Q1
        outlier_point = 1.5 * IQR
        rows.extend(df[(df[col] < Q1 - outlier_point)|(df[col] > Q3 + outlier_point)].index)
    for r, c in Counter(rows).items():
        if c >= n: will_drop_train.append(r)
    return will_drop_train

In [5]:
will_drop_train = outlier_detection_train(data, 5 ,data.select_dtypes(["float", "int"]).columns)
will_drop_train

[338,
 1390,
 6469,
 7038,
 1936,
 3317,
 3980,
 4762,
 6509,
 7007,
 7065,
 7294,
 7689,
 7957,
 8064]

In [6]:
data.drop(will_drop_train, inplace = True, axis = 0)

## 새로운 feature 생성

### 총 사용금액, 그리고 사용한 금액에 따라 poor, middle, rich로 분류

In [7]:
data["Total"] = data["RoomService"] + data["FoodCourt"] + data["ShoppingMall"] + data["Spa"] +\
data["VRDeck"] 
data["RichPoor"] = data["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

test["Total"] = test["RoomService"] + test["FoodCourt"] + test["ShoppingMall"] + test["Spa"] +\
test["VRDeck"] 
test["RichPoor"] = test["Total"].apply(lambda x: "poor" if x < 5000\
                                       else ("middle" if x>=5000 and x<20000 else "rich"))

### 그룹여행객 여부

In [8]:
data["GroupId"] = data["PassengerId"].apply(lambda x: x.split("_")[0])
test["GroupId"] = test["PassengerId"].apply(lambda x: x.split("_")[0])
data["GroupNo"] = data["PassengerId"].apply(lambda x: x.split("_")[1])
test["GroupNo"] = test["PassengerId"].apply(lambda x: x.split("_")[1])

train_g = data[data["GroupId"].duplicated()]["GroupId"]
test_g = test[test["GroupId"].duplicated()]["GroupId"]
data["Group"] = data["GroupId"].apply(lambda x: x in train_g.values)
test["Group"] = test["GroupId"].apply(lambda x: x in test_g.values)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8678 entries, 0 to 8692
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8678 non-null   object 
 1   HomePlanet    8678 non-null   object 
 2   CryoSleep     8678 non-null   bool   
 3   Cabin1        8575 non-null   object 
 4   Cabin2        8575 non-null   float64
 5   Combi         8575 non-null   object 
 6   Cabin3        8575 non-null   object 
 7   Cabin         8575 non-null   object 
 8   Destination   8678 non-null   object 
 9   Age           8678 non-null   int64  
 10  VIP           8678 non-null   bool   
 11  RoomService   8678 non-null   int64  
 12  FoodCourt     8678 non-null   int64  
 13  ShoppingMall  8678 non-null   int64  
 14  Spa           8678 non-null   int64  
 15  VRDeck        8678 non-null   int64  
 16  Name          8478 non-null   object 
 17  Transported   8678 non-null   bool   
 18  Total         8678 non-null 

### 나이브 베어스를 활용한 이름을 통한 성별 분류

In [10]:
# Train_Data
names_train_data = []
for n in data["Name"]:
    n = str(n)
    a = n.split()
    names_train_data.append(a[0])

#### 훈련셋

In [11]:
# 이름과 성 분리
names_train_data = []
for i in data["Name"]:
    i = str(i)
    a = i.split()
    names_train_data.append(a[0])

In [12]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [13]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [14]:
names_train_data

['Maham',
 'Juanna',
 'Altark',
 'Solam',
 'Willy',
 'Sandie',
 'Billex',
 'Candra',
 'Andona',
 'Erraiam',
 'Altardr',
 'Wezena',
 'Berers',
 'Reney',
 'Elle',
 'Justie',
 'Flats',
 'Carry',
 'Alus',
 'Lyde',
 'Philda',
 'Almary',
 'Glendy',
 'Mollen',
 'Breney',
 'Mael',
 'Terta',
 'nan',
 'Penton',
 'Karard',
 'Anyoni',
 'Ceros',
 'Ginia',
 'Coobix',
 'Cinets',
 'Dontch',
 'Ziba',
 'Luse',
 'Marina',
 'Loise',
 'Jorgie',
 'Margia',
 'Ankalik',
 'Jodye',
 'Kayne',
 'Cassa',
 'Zelowl',
 'Mass',
 'Sony',
 'Vivia',
 'Elaney',
 'Elson',
 'Okulas',
 'Instab',
 'Zinoces',
 'Warry',
 'Shanya',
 'Sterry',
 'nan',
 'Colatz',
 'Diandy',
 'Ninaha',
 'Celine',
 'Velyne',
 'Cinst',
 'nan',
 'Meremy',
 'Nelly',
 'Thell',
 'Gorn',
 'Aldibah',
 'Conk',
 'Pon',
 'Spuri',
 'Dellie',
 'Totse',
 'Eaturs',
 'nan',
 'Coren',
 'Furudah',
 'Jodye',
 'Stmeal',
 'Heremy',
 'Deanne',
 'Tinez',
 'Gracy',
 'Stald',
 'Tiney',
 'Alchium',
 'Doria',
 'Leence',
 'Aliey',
 'Thewis',
 'Book',
 'Ritany',
 'Arlen',
 'Sh

In [15]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [16]:
# 성별 feature 생성
names_gender = []
for i in names_train_data:
    names_gender.append(classifier.classify(gender_features(i)))

# create new column called 'gender'    
data["Gender"] = names_gender

In [17]:
data.Gender[data.Name.isna()] = 'female'

In [18]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

#### 테스트셋

In [19]:
# 이름과 성 분리
names_test_data = []
for i in test["Name"]:
    i = str(i)
    a = i.split()
    names_test_data.append(a[0])

In [20]:
# NLTK의 names 파일을 활용하여 이름을 여성과 남성으로 분리
labeled_names = [(name, "female") for name in names.words("female.txt")] + \
[(name, "male") for name in names.words("male.txt")]
random.shuffle(labeled_names)

In [21]:
# 나이브 베어스 모델 학습
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
classifier = NaiveBayesClassifier.train(featuresets)

In [22]:
# 이름의 마지막 단어 가져오는 함수
def gender_features(word):
    return {'last_letter': word[-1]}

In [23]:
# 성별 feature 생성
names_gender = []
for i in names_test_data:
    names_gender.append(classifier.classify(gender_features(i)))

In [24]:
test["Gender"] = pd.Series(names_gender)

In [25]:
test.Gender[test.Name.isna()] = 'female'

In [26]:
data.Gender[data.Name.isna()].unique()

array(['female'], dtype=object)

## Cabin 결측값들 제거

In [27]:
data.dropna(axis=0, inplace=True)

## 필요없는 features 제거

In [28]:
target = data['Transported']
data = data.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId",\
                  "GroupNo", "Transported"], axis = 1)
test = test.drop(["PassengerId", "Name", "Cabin", "Combi", "Total", "GroupId",\
                  "GroupNo", "Transported"], axis = 1)


## 째려보기

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8375 non-null   object 
 1   CryoSleep     8375 non-null   bool   
 2   Cabin1        8375 non-null   object 
 3   Cabin2        8375 non-null   float64
 4   Cabin3        8375 non-null   object 
 5   Destination   8375 non-null   object 
 6   Age           8375 non-null   int64  
 7   VIP           8375 non-null   bool   
 8   RoomService   8375 non-null   int64  
 9   FoodCourt     8375 non-null   int64  
 10  ShoppingMall  8375 non-null   int64  
 11  Spa           8375 non-null   int64  
 12  VRDeck        8375 non-null   int64  
 13  RichPoor      8375 non-null   object 
 14  Group         8375 non-null   bool   
 15  Gender        8375 non-null   object 
dtypes: bool(3), float64(1), int64(6), object(6)
memory usage: 940.6+ KB


In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   object 
 1   CryoSleep     4277 non-null   bool   
 2   Cabin1        4214 non-null   object 
 3   Cabin2        4214 non-null   float64
 4   Cabin3        4214 non-null   object 
 5   Destination   4277 non-null   object 
 6   Age           4277 non-null   int64  
 7   VIP           4277 non-null   bool   
 8   RoomService   4277 non-null   int64  
 9   FoodCourt     4277 non-null   int64  
 10  ShoppingMall  4277 non-null   int64  
 11  Spa           4277 non-null   int64  
 12  VRDeck        4277 non-null   int64  
 13  RichPoor      4277 non-null   object 
 14  Group         4277 non-null   bool   
 15  Gender        4277 non-null   object 
dtypes: bool(3), float64(1), int64(6), object(6)
memory usage: 447.0+ KB


## 원핫인코딩

### boolean 타입 피처들 object로 캐스팅

In [31]:
bool_data = [column for column in data.select_dtypes(["bool"])]

bool_data

['CryoSleep', 'VIP', 'Group']

In [32]:
data["VIP"] = data["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["CryoSleep"] = data["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
data["Group"] = data["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")

test["VIP"] = test["VIP"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test["CryoSleep"] = test["CryoSleep"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")
test["Group"] = test["Group"].replace(to_replace = [False, True], 
                                  value = ["No", "Yes"]).astype("object")

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8375 non-null   object 
 1   CryoSleep     8375 non-null   object 
 2   Cabin1        8375 non-null   object 
 3   Cabin2        8375 non-null   float64
 4   Cabin3        8375 non-null   object 
 5   Destination   8375 non-null   object 
 6   Age           8375 non-null   int64  
 7   VIP           8375 non-null   object 
 8   RoomService   8375 non-null   int64  
 9   FoodCourt     8375 non-null   int64  
 10  ShoppingMall  8375 non-null   int64  
 11  Spa           8375 non-null   int64  
 12  VRDeck        8375 non-null   int64  
 13  RichPoor      8375 non-null   object 
 14  Group         8375 non-null   object 
 15  Gender        8375 non-null   object 
dtypes: float64(1), int64(6), object(9)
memory usage: 1.1+ MB


In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   object 
 1   CryoSleep     4277 non-null   object 
 2   Cabin1        4214 non-null   object 
 3   Cabin2        4214 non-null   float64
 4   Cabin3        4214 non-null   object 
 5   Destination   4277 non-null   object 
 6   Age           4277 non-null   int64  
 7   VIP           4277 non-null   object 
 8   RoomService   4277 non-null   int64  
 9   FoodCourt     4277 non-null   int64  
 10  ShoppingMall  4277 non-null   int64  
 11  Spa           4277 non-null   int64  
 12  VRDeck        4277 non-null   int64  
 13  RichPoor      4277 non-null   object 
 14  Group         4277 non-null   object 
 15  Gender        4277 non-null   object 
dtypes: float64(1), int64(6), object(9)
memory usage: 534.8+ KB


### 더미화

In [35]:
# drop_first 첫번째 범주는 제거하고 더미화
# 다른 범주가 전부 0이면 자동적으로 첫번째 범주가 1인걸 알 수 있기에 사용
df = pd.get_dummies(data, drop_first = True)
t_df = pd.get_dummies(test, drop_first=True)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8375 entries, 0 to 8692
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Cabin2                     8375 non-null   float64
 1   Age                        8375 non-null   int64  
 2   RoomService                8375 non-null   int64  
 3   FoodCourt                  8375 non-null   int64  
 4   ShoppingMall               8375 non-null   int64  
 5   Spa                        8375 non-null   int64  
 6   VRDeck                     8375 non-null   int64  
 7   HomePlanet_Europa          8375 non-null   uint8  
 8   HomePlanet_Mars            8375 non-null   uint8  
 9   CryoSleep_Yes              8375 non-null   uint8  
 10  Cabin1_B                   8375 non-null   uint8  
 11  Cabin1_C                   8375 non-null   uint8  
 12  Cabin1_D                   8375 non-null   uint8  
 13  Cabin1_E                   8375 non-null   uint8

In [37]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Cabin2                     4214 non-null   float64
 1   Age                        4277 non-null   int64  
 2   RoomService                4277 non-null   int64  
 3   FoodCourt                  4277 non-null   int64  
 4   ShoppingMall               4277 non-null   int64  
 5   Spa                        4277 non-null   int64  
 6   VRDeck                     4277 non-null   int64  
 7   HomePlanet_Europa          4277 non-null   uint8  
 8   HomePlanet_Mars            4277 non-null   uint8  
 9   CryoSleep_Yes              4277 non-null   uint8  
 10  Cabin1_B                   4277 non-null   uint8  
 11  Cabin1_C                   4277 non-null   uint8  
 12  Cabin1_D                   4277 non-null   uint8  
 13  Cabin1_E                   4277 non-null   uint8

## 스케일링

In [38]:
df.columns

Index(['Cabin2', 'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa',
       'VRDeck', 'HomePlanet_Europa', 'HomePlanet_Mars', 'CryoSleep_Yes',
       'Cabin1_B', 'Cabin1_C', 'Cabin1_D', 'Cabin1_E', 'Cabin1_F', 'Cabin1_G',
       'Cabin1_T', 'Cabin3_S', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'VIP_Yes', 'RichPoor_poor', 'RichPoor_rich',
       'Group_Yes', 'Gender_male'],
      dtype='object')

In [39]:
# 스케일링
col = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [40]:
data_scaled(df, col)
data_scaled(t_df, col)

,Cabin2,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_Yes,...,Cabin1_G,Cabin1_T,Cabin3_S,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_Yes,RichPoor_poor,RichPoor_rich,Group_Yes,Gender_male
0,3.0,-0.115676,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,0,0,1,...,1,0,1,0,1,0,1,0,0,0
1,4.0,-0.685906,-0.357297,-0.277847,-0.312136,2.287237,-0.246683,0,0,0,...,0,0,1,0,1,0,1,0,0,0
2,0.0,0.169439,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,1,0,1,...,0,0,1,0,0,0,1,0,0,0
3,1.0,0.668390,-0.357297,4.121036,-0.312136,-0.103990,0.226621,1,0,0,...,0,0,1,0,1,0,0,0,0,1
4,5.0,-0.614627,-0.340684,-0.283807,0.832025,-0.267810,-0.246683,0,0,0,...,0,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1496.0,0.383275,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,0,0,1,...,1,0,1,0,1,0,1,0,1,1
4273,NaN,0.953505,-0.357297,0.277063,-0.281505,-0.258759,-0.130177,0,0,0,...,0,0,0,0,1,0,1,0,0,0
4274,296.0,-0.115676,-0.357297,-0.283807,-0.312136,-0.267810,-0.246683,0,1,1,...,0,0,0,0,0,0,1,0,0,1
4275,297.0,-0.115676,-0.357297,1.490845,-0.312136,-0.267810,0.176459,1,0,0,...,0,0,0,0,1,0,1,0,0,1


# 데이터셋 분리

In [41]:
X_train, X_test, y_train, y_test = train_test_split(df, target, shuffle = True, random_state = 109)
X_train, X_val, y_train, y_val = train_test_split(df, target, shuffle = True,\
                                                  test_size=0.15, random_state = 109)

## XGBoost(HyperOPT)

### 모델 찾기

In [42]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4),
                   'n_estimators': hp.quniform('n_estimators', 50, 500, 25)}

In [43]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def xgb_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=int(search_space['n_estimators']),
                                max_depth=int(search_space['max_depth']),
                                min_child_weight=int(search_space['min_child_weight']),
                                colsample_bytree=search_space['colsample_bytree'],
                                learning_rate=search_space['learning_rate'],
                                gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=109)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train, y_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [44]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=xgb_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.88881	validation_1-auc:0.85489                                                                  
[1]	validation_0-auc:0.90021	validation_1-auc:0.86164                                                                  
[2]	validation_0-auc:0.91230	validation_1-auc:0.86969                                                                  
[3]	validation_0-auc:0.91526	validation_1-auc:0.87141                                                                  
[4]	validation_0-auc:0.91886	validation_1-auc:0.87373                                                                  
[5]	validation_0-auc:0.92318	validation_1-auc:0.87672                                                                  
[6]	validation_0-auc:0.92629	validation_1-auc:0.87818                                                                  
[7]	validation_0-auc:0.92985	validation_1-auc:0.87979                                                                  
[8]	validation_0-auc:0.93284	validation_

[15]	validation_0-auc:0.94084	validation_1-auc:0.89965                                                                 
[16]	validation_0-auc:0.94194	validation_1-auc:0.89979                                                                 
[17]	validation_0-auc:0.94273	validation_1-auc:0.90010                                                                 
[18]	validation_0-auc:0.94453	validation_1-auc:0.90036                                                                 
[19]	validation_0-auc:0.94536	validation_1-auc:0.90028                                                                 
[20]	validation_0-auc:0.94604	validation_1-auc:0.90111                                                                 
[21]	validation_0-auc:0.94742	validation_1-auc:0.90137                                                                 
[22]	validation_0-auc:0.94821	validation_1-auc:0.90110                                                                 
[23]	validation_0-auc:0.94854	validation

[31]	validation_0-auc:0.95193	validation_1-auc:0.90309                                                                 
[32]	validation_0-auc:0.95216	validation_1-auc:0.90289                                                                 
[33]	validation_0-auc:0.95319	validation_1-auc:0.90301                                                                 
[34]	validation_0-auc:0.95332	validation_1-auc:0.90307                                                                 
[35]	validation_0-auc:0.95387	validation_1-auc:0.90302                                                                 
[36]	validation_0-auc:0.95448	validation_1-auc:0.90292                                                                 
[37]	validation_0-auc:0.95550	validation_1-auc:0.90348                                                                 
[38]	validation_0-auc:0.95624	validation_1-auc:0.90340                                                                 
[39]	validation_0-auc:0.95669	validation

[6]	validation_0-auc:0.92395	validation_1-auc:0.89196                                                                  
[7]	validation_0-auc:0.92644	validation_1-auc:0.89410                                                                  
[8]	validation_0-auc:0.92927	validation_1-auc:0.89479                                                                  
[9]	validation_0-auc:0.93099	validation_1-auc:0.89593                                                                  
[10]	validation_0-auc:0.93308	validation_1-auc:0.89592                                                                 
[11]	validation_0-auc:0.93646	validation_1-auc:0.89782                                                                 
[12]	validation_0-auc:0.93768	validation_1-auc:0.89872                                                                 
[13]	validation_0-auc:0.93896	validation_1-auc:0.89964                                                                 
[14]	validation_0-auc:0.93975	validation

[30]	validation_0-auc:0.95191	validation_1-auc:0.88979                                                                 
[31]	validation_0-auc:0.95264	validation_1-auc:0.89042                                                                 
[32]	validation_0-auc:0.95302	validation_1-auc:0.89010                                                                 
[33]	validation_0-auc:0.95349	validation_1-auc:0.88966                                                                 
[34]	validation_0-auc:0.95367	validation_1-auc:0.88957                                                                 
[35]	validation_0-auc:0.95418	validation_1-auc:0.89009                                                                 
[36]	validation_0-auc:0.95437	validation_1-auc:0.89034                                                                 
[37]	validation_0-auc:0.95466	validation_1-auc:0.89032                                                                 
[38]	validation_0-auc:0.95544	validation

...

[29]	validation_0-auc:0.93624	validation_1-auc:0.89519                                                                 
[30]	validation_0-auc:0.93715	validation_1-auc:0.89513                                                                 
[31]	validation_0-auc:0.93742	validation_1-auc:0.89477                                                                 
[32]	validation_0-auc:0.93790	validation_1-auc:0.89505                                                                 
[33]	validation_0-auc:0.93791	validation_1-auc:0.89556                                                                 
[34]	validation_0-auc:0.93804	validation_1-auc:0.89596                                                                 
[35]	validation_0-auc:0.93876	validation_1-auc:0.89661                                                                 
[36]	validation_0-auc:0.93923	validation_1-auc:0.89676                                                                 
[37]	validation_0-auc:0.93941	validation

[97]	validation_0-auc:0.95446	validation_1-auc:0.90255                                                                 
[98]	validation_0-auc:0.95441	validation_1-auc:0.90266                                                                 
[99]	validation_0-auc:0.95474	validation_1-auc:0.90273                                                                 
[100]	validation_0-auc:0.95480	validation_1-auc:0.90274                                                                
[101]	validation_0-auc:0.95504	validation_1-auc:0.90284                                                                
[102]	validation_0-auc:0.95556	validation_1-auc:0.90306                                                                
[103]	validation_0-auc:0.95557	validation_1-auc:0.90311                                                                
[104]	validation_0-auc:0.95554	validation_1-auc:0.90312                                                                
[105]	validation_0-auc:0.95554	validatio

[3]	validation_0-auc:0.91246	validation_1-auc:0.86537                                                                  
[4]	validation_0-auc:0.91643	validation_1-auc:0.87191                                                                  
[5]	validation_0-auc:0.91898	validation_1-auc:0.87295                                                                  
[6]	validation_0-auc:0.92066	validation_1-auc:0.87607                                                                  
[7]	validation_0-auc:0.92358	validation_1-auc:0.87876                                                                  
[8]	validation_0-auc:0.92417	validation_1-auc:0.88117                                                                  
[9]	validation_0-auc:0.92204	validation_1-auc:0.88037                                                                  
[10]	validation_0-auc:0.92179	validation_1-auc:0.87812                                                                 
[11]	validation_0-auc:0.92387	validation

[71]	validation_0-auc:0.94996	validation_1-auc:0.89790                                                                 
[72]	validation_0-auc:0.95010	validation_1-auc:0.89796                                                                 
[73]	validation_0-auc:0.95023	validation_1-auc:0.89825                                                                 
[74]	validation_0-auc:0.95035	validation_1-auc:0.89838                                                                 
[75]	validation_0-auc:0.95051	validation_1-auc:0.89864                                                                 
[76]	validation_0-auc:0.95055	validation_1-auc:0.89875                                                                 
[77]	validation_0-auc:0.95053	validation_1-auc:0.89871                                                                 
[78]	validation_0-auc:0.95088	validation_1-auc:0.89877                                                                 
[79]	validation_0-auc:0.95085	validation

[139]	validation_0-auc:0.95807	validation_1-auc:0.90262                                                                
[140]	validation_0-auc:0.95818	validation_1-auc:0.90262                                                                
[141]	validation_0-auc:0.95820	validation_1-auc:0.90258                                                                
[142]	validation_0-auc:0.95826	validation_1-auc:0.90260                                                                
[143]	validation_0-auc:0.95829	validation_1-auc:0.90266                                                                
[144]	validation_0-auc:0.95829	validation_1-auc:0.90266                                                                
[145]	validation_0-auc:0.95834	validation_1-auc:0.90268                                                                
[146]	validation_0-auc:0.95834	validation_1-auc:0.90261                                                                
[147]	validation_0-auc:0.95843	validatio

### 모델 학습

In [45]:
xgb_clf = xgb.XGBClassifier(learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']),
                            min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5),
                            gamma=round(best['gamma']),
                            n_estimators=int(best['n_estimators']),
                            random_state=109)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.69703,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=2, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.14311, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=4, missing=nan,
              monotone_constraints='()', n_estimators=325, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

### 모델 평가

In [46]:
def score_funcs(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차행렬 \n', confusion)
    print('정확도 : {0:.4f}, 정밀도: {1:.4f}, \
         재현율 : {2:.4f}, F1 : {3:.4f}, AUC : {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [47]:
tr_xgb_clf_pred = xgb_clf.predict(X_train)
tr_xgb_clf_proba = xgb_clf.predict_proba(X_train)[:,1]
t_xgb_clf_pred = xgb_clf.predict(X_test)
t_xgb_clf_proba = xgb_clf.predict_proba(X_test)[:,1]
v_xgb_clf_pred = xgb_clf.predict(X_val)
v_xgb_clf_proba = xgb_clf.predict_proba(X_val)[:,1]

print('훈련셋 평가 지표')
score_funcs(y_train,tr_xgb_clf_pred,tr_xgb_clf_proba)
print('\n테스트셋 평가 지표')
score_funcs(y_test,t_xgb_clf_pred,t_xgb_clf_proba)
print('\n검증셋 평가 지표')
score_funcs(y_val,v_xgb_clf_pred,v_xgb_clf_proba)

훈련셋 평가 지표
오차행렬 
 [[3085  443]
 [ 426 3164]]
정확도 : 0.8779, 정밀도: 0.8772,          재현율 : 0.8813, F1 : 0.8793, AUC : 0.9579

테스트셋 평가 지표
오차행렬 
 [[860 175]
 [175 884]]
정확도 : 0.8329, 정밀도: 0.8347,          재현율 : 0.8347, F1 : 0.8347, AUC : 0.9253

검증셋 평가 지표
오차행렬 
 [[502 116]
 [128 511]]
정확도 : 0.8059, 정밀도: 0.8150,          재현율 : 0.7997, F1 : 0.8073, AUC : 0.9040


# 예측

In [48]:
t_pred = pd.DataFrame(xgb_clf.predict(t_df))
t_pred.replace({0:False, 1:True}, inplace=True)
t_pred.value_counts()

False    2177
True     2100
dtype: int64

In [49]:
t_pred.to_csv('final')